# Use ATM_Payments_Distribution Notebook Template

In [2]:
from fosforio import snowflake
import pandas as pd
import datetime as dt

In [3]:
!pip freeze | grep pandas

pandas==2.0.3


In [4]:
import seaborn as sns
import subprocess

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

Matplotlib created a temporary cache directory at /tmp/matplotlib-f2s99nm5 because the default path (/home/mosaic-ai/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [5]:
from fosforml import *
from fosforml.constants import MLModelFlavours
import requests

In [6]:
# Import label encoder 
from sklearn import preprocessing 
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from math import sqrt
import numpy as np

In [7]:
# To get snowflake connection object with a default snowflake connection created by the user, if available.
#snowflake.get_connection()

# To get snowflake connection object with a specific connection name
snowflake.get_connection(connection_name="FDC_Banking_FS_SNOWFLAKE")

Exception occurred in getting snowflake connection: 'connectionSources'


In [8]:
# To read a specific dataset published from a snowflake connection
df_original = snowflake.get_dataframe("ATM_TRANSACTION_MASTER_DATA")
df = snowflake.get_dataframe("ATM_TRANSACTION_MASTER_DATA")

In [9]:
to_drop = ['ATM_STATE_DETAILS_STATE','MALE_POPULATION','FEMALE_POPULATION','TOTAL_POPULATION','NO_OF_VETERANS','FOREIGN_BORN','AVG_HOUSEHOLD_SIZE','LANDAREA_SQ_MILE','POPULATION_RATIO','POPULATION_DENSITY','MEDIAN_HOUSE_INCOME']
df.drop(to_drop, inplace=True, axis=1)

In [10]:
df.shape

(312417, 11)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312417 entries, 0 to 312416
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   DATE                     312417 non-null  object 
 1   WEEKDAY_FLAG             312417 non-null  object 
 2   HOLIDAY_FLAG             312417 non-null  object 
 3   BANK_ID                  312417 non-null  int8   
 4   STATE                    312417 non-null  object 
 5   BANK_NAME                312417 non-null  object 
 6   ATM_ID                   312417 non-null  object 
 7   DISPENSED_AMOUNT         312417 non-null  float64
 8   TOTAL_TRANSACTION_COUNT  312417 non-null  float64
 9   DOWNTIME_IN_MINS         312417 non-null  float64
 10  ATM_MAX_CAPACITY         312417 non-null  float64
dtypes: float64(4), int8(1), object(6)
memory usage: 24.1+ MB


In [12]:
df.head()

,DATE,WEEKDAY_FLAG,HOLIDAY_FLAG,BANK_ID,STATE,BANK_NAME,ATM_ID,DISPENSED_AMOUNT,TOTAL_TRANSACTION_COUNT,DOWNTIME_IN_MINS,ATM_MAX_CAPACITY
0,2023-01-29,Y,N,3,Maine,Citibank,TBH000274025,225180.0,90.0,0.0,2860000.0
1,2023-01-30,Y,N,3,Maine,Citibank,TBH000274025,262440.0,117.0,0.0,2860000.0
2,2023-01-31,Y,N,3,Maine,Citibank,TBH000274025,561150.0,160.0,0.0,2860000.0
3,2023-02-01,Y,N,3,Maine,Citibank,TBH000274025,437220.0,129.0,0.0,2860000.0
4,2023-02-02,Y,N,3,Maine,Citibank,TBH000274025,360900.0,119.0,0.0,2860000.0


In [13]:
df.columns

Index(['DATE', 'WEEKDAY_FLAG', 'HOLIDAY_FLAG', 'BANK_ID', 'STATE', 'BANK_NAME',
       'ATM_ID', 'DISPENSED_AMOUNT', 'TOTAL_TRANSACTION_COUNT',
       'DOWNTIME_IN_MINS', 'ATM_MAX_CAPACITY'],
      dtype='object')

In [14]:
df.isnull().sum()

DATE                       0
WEEKDAY_FLAG               0
HOLIDAY_FLAG               0
BANK_ID                    0
STATE                      0
BANK_NAME                  0
ATM_ID                     0
DISPENSED_AMOUNT           0
TOTAL_TRANSACTION_COUNT    0
DOWNTIME_IN_MINS           0
ATM_MAX_CAPACITY           0
dtype: int64

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312417 entries, 0 to 312416
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   DATE                     312417 non-null  object 
 1   WEEKDAY_FLAG             312417 non-null  object 
 2   HOLIDAY_FLAG             312417 non-null  object 
 3   BANK_ID                  312417 non-null  int8   
 4   STATE                    312417 non-null  object 
 5   BANK_NAME                312417 non-null  object 
 6   ATM_ID                   312417 non-null  object 
 7   DISPENSED_AMOUNT         312417 non-null  float64
 8   TOTAL_TRANSACTION_COUNT  312417 non-null  float64
 9   DOWNTIME_IN_MINS         312417 non-null  float64
 10  ATM_MAX_CAPACITY         312417 non-null  float64
dtypes: float64(4), int8(1), object(6)
memory usage: 24.1+ MB


In [16]:
df['DATE'] = pd.to_datetime(df['DATE'])
df['HOUR'] = df['DATE'].dt.hour
df['DAYOFWEEK'] = df['DATE'].dt.dayofweek
df['QUARTER'] = df['DATE'].dt.quarter
df['MONTH'] = df['DATE'].dt.month
df['YEAR'] = df['DATE'].dt.year
df['DAYOFYEAR'] = df['DATE'].dt.dayofyear
df['DAYOFMONTH'] = df['DATE'].dt.day
df['DATE'] = pd.to_datetime(df['DATE']).dt.strftime("%Y-%m-%d %H:%M:%S.%f")

In [17]:
df['WEEKDAY_FLAG'] = df['WEEKDAY_FLAG'].map({'Y': 1, 'N': 0})
df['HOLIDAY_FLAG'] = df['HOLIDAY_FLAG'].map({'Y': 1, 'N': 0})

In [18]:
df.head()

,DATE,WEEKDAY_FLAG,HOLIDAY_FLAG,BANK_ID,STATE,BANK_NAME,ATM_ID,DISPENSED_AMOUNT,TOTAL_TRANSACTION_COUNT,DOWNTIME_IN_MINS,ATM_MAX_CAPACITY,HOUR,DAYOFWEEK,QUARTER,MONTH,YEAR,DAYOFYEAR,DAYOFMONTH
0,2023-01-29 00:00:00.000000,1,0,3,Maine,Citibank,TBH000274025,225180.0,90.0,0.0,2860000.0,0,6,1,1,2023,29,29
1,2023-01-30 00:00:00.000000,1,0,3,Maine,Citibank,TBH000274025,262440.0,117.0,0.0,2860000.0,0,0,1,1,2023,30,30
2,2023-01-31 00:00:00.000000,1,0,3,Maine,Citibank,TBH000274025,561150.0,160.0,0.0,2860000.0,0,1,1,1,2023,31,31
3,2023-02-01 00:00:00.000000,1,0,3,Maine,Citibank,TBH000274025,437220.0,129.0,0.0,2860000.0,0,2,1,2,2023,32,1
4,2023-02-02 00:00:00.000000,1,0,3,Maine,Citibank,TBH000274025,360900.0,119.0,0.0,2860000.0,0,3,1,2,2023,33,2


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312417 entries, 0 to 312416
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   DATE                     312417 non-null  object 
 1   WEEKDAY_FLAG             312417 non-null  int64  
 2   HOLIDAY_FLAG             312417 non-null  int64  
 3   BANK_ID                  312417 non-null  int8   
 4   STATE                    312417 non-null  object 
 5   BANK_NAME                312417 non-null  object 
 6   ATM_ID                   312417 non-null  object 
 7   DISPENSED_AMOUNT         312417 non-null  float64
 8   TOTAL_TRANSACTION_COUNT  312417 non-null  float64
 9   DOWNTIME_IN_MINS         312417 non-null  float64
 10  ATM_MAX_CAPACITY         312417 non-null  float64
 11  HOUR                     312417 non-null  int32  
 12  DAYOFWEEK                312417 non-null  int32  
 13  QUARTER                  312417 non-null  int32  
 14  MONT

In [20]:
# how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
df['STATE']= label_encoder.fit_transform(df['STATE'])

In [21]:
# how to understand word labels. 
atm_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
df['ATM_ID']= atm_encoder.fit_transform(df['ATM_ID'])

In [22]:
df.drop(['DATE','BANK_NAME','TOTAL_TRANSACTION_COUNT','DOWNTIME_IN_MINS','ATM_MAX_CAPACITY'], axis=1,inplace=True)

In [23]:
df.columns

Index(['WEEKDAY_FLAG', 'HOLIDAY_FLAG', 'BANK_ID', 'STATE', 'ATM_ID',
       'DISPENSED_AMOUNT', 'HOUR', 'DAYOFWEEK', 'QUARTER', 'MONTH', 'YEAR',
       'DAYOFYEAR', 'DAYOFMONTH'],
      dtype='object')

In [24]:
features = ['WEEKDAY_FLAG', 'HOLIDAY_FLAG', 'BANK_ID', 'STATE', 'ATM_ID',
    'HOUR', 'DAYOFWEEK', 'QUARTER', 'MONTH', 'YEAR',
       'DAYOFYEAR', 'DAYOFMONTH']

In [25]:
print (df.shape)
df = df[df['DISPENSED_AMOUNT'] > 0]
print (df.shape)

(312417, 13)
(257564, 13)


In [26]:
#To be used for Experimentation
df.to_csv('/data/Output/transaction_processed.csv',index=False)

In [27]:
X = df[features]
y = df['DISPENSED_AMOUNT']

In [28]:
X.head()

,WEEKDAY_FLAG,HOLIDAY_FLAG,BANK_ID,STATE,ATM_ID,HOUR,DAYOFWEEK,QUARTER,MONTH,YEAR,DAYOFYEAR,DAYOFMONTH
0,1,0,3,18,299,0,6,1,1,2023,29,29
1,1,0,3,18,299,0,0,1,1,2023,30,30
2,1,0,3,18,299,0,1,1,1,2023,31,31
3,1,0,3,18,299,0,2,1,2,2023,32,1
4,1,0,3,18,299,0,3,1,2,2023,33,2


In [29]:
y

0         225180.0
1         262440.0
2         561150.0
3         437220.0
4         360900.0
            ...   
312412    195300.0
312413    245160.0
312414    222480.0
312415    198360.0
312416    314190.0
Name: DISPENSED_AMOUNT, Length: 257564, dtype: float64

In [30]:
# Split the data into training and test sets. (0.75, 0.25) split.
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25)

In [31]:
print(f'Total # of sample in whole dataset: {len(X)}')
print("*****"*10)
print(f'Total # of sample in train dataset: {len(X_train)}')
print(f'Shape of X_train: {X_train.shape}')
print("*****"*10)
print(f'Total # of sample in test dataset: {len(X_test)}')
print(f'Shape of X_test: {X_test.shape}')

Total # of sample in whole dataset: 257564
**************************************************
Total # of sample in train dataset: 193173
Shape of X_train: (193173, 12)
**************************************************
Total # of sample in test dataset: 64391
Shape of X_test: (64391, 12)


# Decision Tree

In [32]:
tree = DecisionTreeRegressor(max_depth=4,max_features=4)

In [33]:
tree.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=4, max_features=4)

In [34]:
predictions = tree.predict(X_test)

In [35]:
print('Mean Absolute Error:', mean_absolute_error(y_test,predictions))
print('Mean Squared Error:', mean_squared_error(y_test,predictions))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test,predictions)))
print('r2_score:', r2_score(y_test,predictions))

Mean Absolute Error: 191016.90135842178
Mean Squared Error: 59442985494.8596
Root Mean Squared Error: 243809.3220015584
r2_score: 0.14414564067234314


In [36]:
tree.feature_importances_

array([0.        , 0.        , 0.05196462, 0.02615255, 0.59195933,
       0.        , 0.00854072, 0.        , 0.01607731, 0.00851988,
       0.01300294, 0.28378265])

In [37]:
pd.Series(tree.feature_importances_,index=features).sort_values(ascending=False)

ATM_ID          0.591959
DAYOFMONTH      0.283783
BANK_ID         0.051965
STATE           0.026153
MONTH           0.016077
DAYOFYEAR       0.013003
DAYOFWEEK       0.008541
YEAR            0.008520
WEEKDAY_FLAG    0.000000
HOLIDAY_FLAG    0.000000
HOUR            0.000000
QUARTER         0.000000
dtype: float64

In [38]:
param_grid = [{"max_depth":[3,4,5, None], "max_features":[3,4,5,6,7]}]

In [39]:
gs = GridSearchCV(estimator=DecisionTreeRegressor(random_state=123),param_grid = param_grid,cv=10)

In [ ]:
gs.fit(X_train, y_train)

In [ ]:
gs.cv_results_['params']

In [ ]:
gs.cv_results_['rank_test_score']

In [ ]:
gs.best_estimator_

In [ ]:
#Build Tree using Best parameters

tree1 = DecisionTreeRegressor(max_features=7, random_state=123)
tree1.fit(X_train, y_train)
predictions = tree1.predict(X_test)
y_pred = predictions

In [ ]:
X_train.columns

In [ ]:
print('Mean Absolute Error:', mean_absolute_error(y_test,predictions))
print('Mean Squared Error:', mean_squared_error(y_test,predictions))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test,predictions)))
print('r2_score:', r2_score(y_test,predictions))

In [47]:
payload_dict = {'DATE': '2023-01-29', 'WEEKDAY_FLAG': 'Y', 'HOLIDAY_FLAG': 'N', 'BANK_ID': 3, 'STATE': 'Maine', 'BANK_NAME': 'Citibank', 'ATM_ID': 'TBH000274025', 'DISPENSED_AMOUNT': 225180.0, 'TOTAL_TRANSACTION_COUNT': 90.0, 'DOWNTIME_IN_MINS': 0.0, 'ATM_MAX_CAPACITY': 2860000.0}

In [48]:
eval(str(payload_dict))

{'DATE': '2023-01-29',
 'WEEKDAY_FLAG': 'Y',
 'HOLIDAY_FLAG': 'N',
 'BANK_ID': 3,
 'STATE': 'Maine',
 'BANK_NAME': 'Citibank',
 'ATM_ID': 'TBH000274025',
 'DISPENSED_AMOUNT': 225180.0,
 'TOTAL_TRANSACTION_COUNT': 90.0,
 'DOWNTIME_IN_MINS': 0.0,
 'ATM_MAX_CAPACITY': 2860000.0}

In [49]:
pd.DataFrame(payload_dict,index=[0])

,DATE,WEEKDAY_FLAG,HOLIDAY_FLAG,BANK_ID,STATE,BANK_NAME,ATM_ID,DISPENSED_AMOUNT,TOTAL_TRANSACTION_COUNT,DOWNTIME_IN_MINS,ATM_MAX_CAPACITY
0,2023-01-29,Y,N,3,Maine,Citibank,TBH000274025,225180.0,90.0,0.0,2860000.0


In [50]:
@scoring_func
def score(model, request):
    payload_dict = request.json["payload"]
    df = pd.DataFrame(payload_dict,index=[0])
    
    df['DATE'] = pd.to_datetime(df['DATE'])
    df['HOUR'] = df['DATE'].dt.hour
    df['DAYOFWEEK'] = df['DATE'].dt.dayofweek
    df['QUARTER'] = df['DATE'].dt.quarter
    df['MONTH'] = df['DATE'].dt.month
    df['YEAR'] = df['DATE'].dt.year
    df['DAYOFYEAR'] = df['DATE'].dt.dayofyear
    df['DAYOFMONTH'] = df['DATE'].dt.day
    df['DATE'] = pd.to_datetime(df['DATE']).dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    df['WEEKDAY_FLAG'] = df['WEEKDAY_FLAG'].map({'Y': 1, 'N': 0})
    df['HOLIDAY_FLAG'] = df['HOLIDAY_FLAG'].map({'Y': 1, 'N': 0})

    df['STATE']= label_encoder.transform(df['STATE'])
    df['ATM_ID']= atm_encoder.transform(df['ATM_ID'])
    
    if 'TOTAL_TRANSACTION_COUNT' in df.columns:
        df.drop(['TOTAL_TRANSACTION_COUNT'] ,axis=1, inplace=True)
    
    if 'DOWNTIME_IN_MINS' in df.columns:
        df.drop(['DOWNTIME_IN_MINS'] ,axis=1, inplace=True)
        
    if 'ATM_MAX_CAPACITY' in df.columns:
        df.drop(['ATM_MAX_CAPACITY'] ,axis=1, inplace=True)

    df.drop(['DATE','BANK_NAME'], axis=1,inplace=True)
    
    features = ['WEEKDAY_FLAG', 'HOLIDAY_FLAG', 'BANK_ID', 'STATE', 'ATM_ID','HOUR', 'DAYOFWEEK', 
                'QUARTER', 'MONTH', 'YEAR','DAYOFYEAR', 'DAYOFMONTH']

    data = df[features]

    y_pred = model.predict(data)[0]
    return y_pred

In [51]:
df_original_1 = df_original.copy()
payload  = df_original.iloc[0].to_dict()
#payload1  = df_original.iloc[0].to_dict()
payload

{'DATE': '2023-01-29',
 'WEEKDAY_FLAG': 'Y',
 'HOLIDAY_FLAG': 'N',
 'BANK_ID': 3,
 'STATE': 'Maine',
 'BANK_NAME': 'Citibank',
 'ATM_ID': 'TBH000274025',
 'DISPENSED_AMOUNT': 225180.0,
 'TOTAL_TRANSACTION_COUNT': 90.0,
 'DOWNTIME_IN_MINS': 0.0,
 'ATM_MAX_CAPACITY': 2860000.0}

In [52]:
print ('{ "payload": ', payload, "}")

{ "payload":  {'DATE': '2023-01-29', 'WEEKDAY_FLAG': 'Y', 'HOLIDAY_FLAG': 'N', 'BANK_ID': 3, 'STATE': 'Maine', 'BANK_NAME': 'Citibank', 'ATM_ID': 'TBH000274025', 'DISPENSED_AMOUNT': 225180.0, 'TOTAL_TRANSACTION_COUNT': 90.0, 'DOWNTIME_IN_MINS': 0.0, 'ATM_MAX_CAPACITY': 2860000.0} }


In [53]:
df_original_1.drop(['TOTAL_TRANSACTION_COUNT','DOWNTIME_IN_MINS','ATM_MAX_CAPACITY'], inplace=True, axis=1)
payload1  = df_original_1.iloc[0].to_dict()

In [54]:
print ('{ "payload": ', payload1, "}")

{ "payload":  {'DATE': '2023-01-29', 'WEEKDAY_FLAG': 'Y', 'HOLIDAY_FLAG': 'N', 'BANK_ID': 3, 'STATE': 'Maine', 'BANK_NAME': 'Citibank', 'ATM_ID': 'TBH000274025', 'DISPENSED_AMOUNT': 225180.0} }


In [55]:
req = requests.Request()
req.json = {"payload":payload1}
#req.json = {"payload":payload}
y_req = req
score(tree, y_req)

361830.05320568156

In [56]:
req = requests.Request()
req.json = {"payload":payload}
y_req = req
score(tree, y_req)

361830.05320568156

In [57]:
y_prediction = pd.Series(y_pred)

In [58]:
y_prediction

0        211320.0
1        776250.0
2        383580.0
3        183600.0
4        443790.0
           ...   
67945    636480.0
67946    705060.0
67947    441900.0
67948    260550.0
67949    508050.0
Length: 67950, dtype: float64

In [59]:
type(X_train), type(X_test), type(y_train), type(y_test), type(y_pred), type(y_prediction)

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.series.Series,
 pandas.core.series.Series,
 numpy.ndarray,
 pandas.core.series.Series)

In [60]:
## registering the model in Fosfor.
model_reg = register_model(tree1,
               score, 
               name="ATM_DispenseAMT_Dtree_Regression", 
               description="ATM Dispense Amount DTree Regression",
               flavour=MLModelFlavours.sklearn,
               model_type="regression",
               init_script="\\n pip install fosforml \\n pip install fosforio[snowflake] \\n pip install seaborn \\n pip install snowflake-connector-python[pandas]",
               y_true=y_test,
               y_pred=y_prediction,
               #prob=y_prob,
               features=X_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train.tolist(),
               y_test=y_test.tolist(),
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               kyd=True, kyd_score = True)

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


# 2nd Version of Dtree model

In [61]:
tree2 = DecisionTreeRegressor(max_features=8, random_state=123)
tree2.fit(X_train, y_train)
predictions = tree2.predict(X_test)
y_pred = predictions
y_prediction = pd.Series(y_pred)


print('Mean Absolute Error:', mean_absolute_error(y_test,predictions))
print('Mean Squared Error:', mean_squared_error(y_test,predictions))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test,predictions)))
print('r2_score:', r2_score(y_test,predictions))

Mean Absolute Error: 2071.1774834437088
Mean Squared Error: 729123308.4635762
Root Mean Squared Error: 27002.283393512782
r2_score: 0.989476592295161


In [62]:
## registering the model in Fosfor.
model_reg = register_model(tree2,
               score, 
               name="ATM_DispenseAMT_Dtree_Regression", 
               description="ATM Dispense Amount DTree Regression",
               flavour=MLModelFlavours.sklearn,
               model_type="regression",
               init_script="\\n pip install fosforml \\n pip install fosforio[snowflake] \\n pip install seaborn \\n pip install snowflake-connector-python[pandas]",
               y_true=y_test,
               y_pred=y_prediction,
               #prob=y_prob,
               features=X_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train.tolist(),
               y_test=y_test.tolist(),
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               kyd=True, kyd_score = True)

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


In [ ]:
from mosaic_utils.ai.file_utils import pickle_loads, pickle_dumps
pickle_dumps(X_train,"/tmp/aa")
pickle_loads("/tmp/aa")

In [ ]:
!pip freeze | grep pandas

# Use Snowflake Data for Model Prediction

In [ ]:
df = snowflake.get_dataframe("ATM_TRANSACTION_MASTER_DATA")

In [ ]:
df = df[df['DISPENSED_AMOUNT'] > 0]

In [ ]:
df.columns

In [ ]:
df_train = df.sample(frac = 0.7)
df_test = df.drop(df_train.index)

In [ ]:
df_train.shape, df_test.shape

In [ ]:
def get_prediction(data):
    df = data.copy()
    to_drop = ['ATM_STATE_DETAILS_STATE','MALE_POPULATION','FEMALE_POPULATION','TOTAL_POPULATION','NO_OF_VETERANS','FOREIGN_BORN','AVG_HOUSEHOLD_SIZE','LANDAREA_SQ_MILE','POPULATION_RATIO','POPULATION_DENSITY','MEDIAN_HOUSE_INCOME']
    df.drop(to_drop, inplace=True, axis=1)
    df['DATE'] = pd.to_datetime(df['DATE'])
    df['HOUR'] = df['DATE'].dt.hour
    df['DAYOFWEEK'] = df['DATE'].dt.dayofweek
    df['QUARTER'] = df['DATE'].dt.quarter
    df['MONTH'] = df['DATE'].dt.month
    df['YEAR'] = df['DATE'].dt.year
    df['DAYOFYEAR'] = df['DATE'].dt.dayofyear
    df['DAYOFMONTH'] = df['DATE'].dt.day
    df['DATE'] = pd.to_datetime(df['DATE']).dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    
    df['WEEKDAY_FLAG'] = df['WEEKDAY_FLAG'].map({'Y': 1, 'N': 0})
    df['HOLIDAY_FLAG'] = df['HOLIDAY_FLAG'].map({'Y': 1, 'N': 0})
    
    df['STATE']= label_encoder.transform(df['STATE'])
    df['ATM_ID']= atm_encoder.transform(df['ATM_ID'])
    
    df.drop(['DATE','BANK_NAME'], axis=1,inplace=True)
    
    features = ['WEEKDAY_FLAG', 'HOLIDAY_FLAG', 'BANK_ID', 'STATE', 'ATM_ID','HOUR', 'DAYOFWEEK', 
                'QUARTER', 'MONTH', 'YEAR','DAYOFYEAR', 'DAYOFMONTH']
    
    X_feat = df[features]
    return tree.predict(X_feat)

In [ ]:
df_train_y  = get_prediction(df_train)
df_test_y  = get_prediction(df_test)

In [ ]:
df_train['PREDICTED_DISPENSE_AMOUNT'] = df_train_y
df_test['PREDICTED_DISPENSE_AMOUNT'] = df_test_y

In [ ]:
df_train['DATE'] = pd.to_datetime(df_train['DATE']).dt.strftime("%Y-%m-%d %H:%M:%S.%f")
df_test['DATE'] = pd.to_datetime(df_test['DATE']).dt.strftime("%Y-%m-%d %H:%M:%S.%f")

In [ ]:
df_train.head()

In [ ]:
df_train.to_csv('/data/Output/transaction_train_output.csv',index=True)
df_test.to_csv('/data/Output/transaction_test_output.csv',index=True)

# Push the output to Snowflake tables

In [ ]:
from snowflake.snowpark import Session
import warnings; warnings.simplefilter('ignore')
import configparser

In [ ]:
#Import all snowflake connection details from template variables.
db_user = 'MANISH'
db_password = 'Password@2023'
db_account = 'ug94937.us-east4.gcp'
db_database =  'FDC_BANKING_FS'
db_role = 'MANISH'
db_warehouse = 'FOSFOR_SOLUTIONS_WH'
db_schema = 'PUBLIC'

In [ ]:
from snowflake.snowpark.session import Session
connection_params = {
    'user': db_user,
    'password': db_password,
    'account': db_account,
    'warehouse': db_warehouse,
    'database': db_database,
    'schema': db_schema,
    'role': db_role
}
session1 = Session.builder.configs(connection_params).create()

In [ ]:
df_train_sf=session1.createDataFrame(
        df_train.values.tolist(),
        schema=df_train.columns.tolist())
df_train_sf.write.mode("overwrite").save_as_table("FDC_BANKING_FS.PUBLIC.ATM_TRANSACTION_TRAIN_OUTPUT")

In [ ]:
df_test_sf=session1.createDataFrame(
        df_test.values.tolist(),
        schema=df_test.columns.tolist())
df_test_sf.write.mode("overwrite").save_as_table("FDC_BANKING_FS.PUBLIC.ATM_TRANSACTION_TEST_OUTPUT")

# Run below query on snowflake to final demo setup

ALTER TABLE FDC_BANKING_FS.PUBLIC.ATM_TRANSACTION_TEST_OUTPUT ADD COLUMN REPORT_DATE DATE
UPDATE TABLE FDC_BANKING_FS.PUBLIC.ATM_TRANSACTION_TEST_OUTPUT
SET REPORT_DATE =  TO_DATE(DATE)

ALTER TABLE FDC_BANKING_FS.PUBLIC.ATM_TRANSACTION_TRAIN_OUTPUT ADD COLUMN REPORT_DATE DATE
UPDATE FDC_BANKING_FS.PUBLIC.ATM_TRANSACTION_TRAIN_OUTPUT
SET REPORT_DATE =  TO_DATE(DATE)